In [1]:
import numpy as np
import pandas as pd
import Linear_Regression as LR
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal

In [2]:
a_none = np.array(None)
data = pd.read_csv('data.csv')
data_train = data[0:500].select_dtypes(include='number')
price = data_train['price']
data_train = data_train.drop(['price', 'Unnamed: 0.1',	'Unnamed: 0'], axis=1)
data_train.insert(0, 'intersection', pd.DataFrame([1]*len(data_train)))
data_train.insert(data_train.shape[1], 'price', price)

data_test = data[500:].select_dtypes(include='number')
price = data_test['price']
data_test = data_test.drop(['price', 'Unnamed: 0.1',	'Unnamed: 0'], axis=1)
data_test.insert(0, 'intersection', pd.DataFrame([1]*len(data_test), index=list(data_test.index)))
data_test.insert(data_test.shape[1], 'price', price)
x_test = data_test.drop('price', axis=1)
y_test = data_test['price']

In [3]:
class Polinomial_regression(LR.LR):

    def __init__(self, Data, grau, select=[], skip=[]):
        Data = np.array(Data)
        self.x_non = Data[:, :-1]
        self.y = Data[:, -1]
        self.x_pol = self.Novas_Var(grau, self.x_non, select=select, skip=skip)
        super().__init__(self.x_pol, self.y)

    def Novas_Var(self, grau, x=a_none, select=[], skip=[]):
        if x.all() == None:
            newx = np.ones([self.x.shape[0], (self.x.shape[1]-1)*grau-(len(skip)*grau-(len(skip))) + 1])
            x = self.x.copy()
            m=0
            for i in range(1, self.x.shape[1]):
                if len(skip) != 0 and i in skip:
                    newx[:, m+1] = x[:,i]
                elif select!=None and i in select:
                    for j in range(1, grau+1):
                        newx[:, m+j] = x[:,i]**j
                    m=m+j
                else:
                    for j in range(1, grau+1):
                        newx[:, m+j] = x[:,i]**j
                    m=m+j
            return newx
        else:
            x_data = np.array(x).copy()
            newx = np.ones([x.shape[0], (x.shape[1]-1)*grau-(len(skip)*grau-(len(skip))) + 1])
            m=0
            for i in range(1, x.shape[1]):
                if len(skip) != 0 and i in skip:
                    newx[:, m+1] = x[:,i]
                elif len(select)!=0 and i in select:
                    for j in range(1, grau+1):
                        newx[:, m+j] = x_data[:,i]**j
                    m=m+j
                else:
                    for j in range(1, grau+1):
                        newx[:, m+j] = x_data[:,i]**j
                    m=m+j
            return newx

In [4]:
# Ortogonalizando os dados do treino
#data_train_o = LR.gram_schmidt(data_train)
pol = Polinomial_regression(data_train, 3, skip=[5])
pol.x = LR.gram_schmidt(pd.DataFrame(pol.x)).to_numpy()
pol.least_square()
pol.MSE()

# Elevando as variáveis do data set
data_test_polin = pol.Novas_Var(3, x_test.to_numpy(), skip=[5])
# Ortogonalizando os dados do treino
data_test_polin = LR.gram_schmidt(pd.DataFrame(data_test_polin)).to_numpy()
y_hat = pol.prediction(data_test_polin)
pol.MSE(y_hat, y_test)

2373005372.8328824